# LU разложение, блочные и ленточные матрицы

## Объявление классов

In [2]:
import numpy as np

In [3]:
class TextBlock:
    def __init__(self, rows):
        assert isinstance(rows, list)
        self.rows = rows
        self.height = len(self.rows)
        self.width = max(map(len,self.rows))
        
    @classmethod
    def from_str(_cls, data):
        assert isinstance(data, str)
        return TextBlock( data.split('\n') )
        
    def format(self, width=None, height=None):
        if width is None: width = self.width
        if height is None: height = self.height
        return [f"{row:{width}}" for row in self.rows]+[' '*width]*(height-self.height)
    
    @staticmethod
    def merge(blocks):
        return [" ".join(row) for row in zip(*blocks)]
    
class Matrix:
    """Общий предок для всех матриц."""
    @property
    def shape(self):
        raise NotImplementedError
    
    @property
    def dtype(self):
        raise NotImplementedError
    
    @property 
    def width(self):
        return self.shape[1]
    
    @property 
    def height(self):
        return self.shape[0]    
        
    def __repr__(self):
        """Возвращает текстовое представление для матрицы."""
        text = [[TextBlock.from_str(f"{self[r,c]}") for c in range(self.width)] for r in range(self.height)]
        width_el = np.array(list(map(lambda row: list(map(lambda el: el.width, row)), text)))
        height_el = np.array(list(map(lambda row: list(map(lambda el: el.height, row)), text)))
        width_column = np.max(width_el, axis=0)
        width_total = np.sum(width_column)
        height_row = np.max(height_el, axis=1)
        result = []
        for r in range(self.height):
            lines = TextBlock.merge(text[r][c].format(width=width_column[c], height=height_row[r]) for c in range(self.width))
            for l in lines:
                result.append(f"| {l} |")
            if len(lines)>0 and len(lines[0])>0 and lines[0][0]=='|' and r<self.height-1:
                result.append(f'| {" "*(width_total+self.width)}|')
        return "\n".join(result)
    
    def empty_like(self, width=None, height=None):
        raise NotImplementedError
    
    def __getitem__(self, key):
        raise NotImplementedError
    
    def __setitem__(self, key, value):
        raise NotImplementedError
        
    def __add__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c] = self[r,c] + other[r,c]
            return matrix
        return NotImplemented
    
    def __sub__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c] = self[r,c] - other[r,c]
            return matrix
        return NotImplemented

    def __mul__(self, other):
        return self.__matmul__(other)
    
    def __matmul__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(other.width):
                    acc = None
                    for k in range(self.width):
                        add = self[r,k]*other[k,c]
                        acc = add if acc is None else acc+add
                    matrix[r,c] = acc
            return matrix
        return NotImplemented
    
    def inverse(self):
        raise NotImplementedError
        
    def invert_element(self, element):
        if isinstance(element, float):
            return 1/element
        if isinstance(element, Fraction):
            return 1/element
        if isinstance(element, Matrix):
            return element.inverse()
        raise TypeError
        

class FullMatrix(Matrix):
    """
    Заполненная матрица с элементами произвольного типа.
    """
    def __init__(self, data):
        """
        Создает объект, хранящий матрицу в виде np.ndarray `data`.
        """
        assert isinstance(data, np.ndarray)
        self.data = data

    def empty_like(self, width=None, height=None):
        dtype = self.data.dtype
        if width is None:
            width = self.data.shape[1]
        if height is None:
            height = self.data.shape[0]       
        data = np.empty((height,width), dtype=dtype)
        return FullMatrix(data)
        
    @classmethod
    def zero(_cls, height, width, default=0):
        """
        Создает матрицу размера `width` x `height` со значениями по умолчанию `default`.
        """
        data = np.empty((height, width), dtype=type(default))
        data[:] = default
        return FullMatrix(data)
                    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def dtype(self):
        return self.data.dtype
        
    def __getitem__(self, key):
        row, column = key
        return self.data[row, column]
    
    def __setitem__(self, key, value):
        row, column = key
        self.data[row, column] = value
        

## Задание

Существует множество типов матриц, удовлетворяющих некоторым дополнительным условиям, для которых многие матричные операции могут быть вычислены быстрее или точнее, чем для матриц произвольного вида. 
В данной лабораторной мы начнем писать библиотеку на Python, которая будет содержать классы, реализующие базовые алгоритмы для работы с основными типами матриц.
Далее приводится исходный код класса `Matrix`, являющегося общим предком для всех матриц, и реализующего логику работы с матрицами общего вида.
Нижеследующий класс `FullMatrix` реализует хранилище для заполненных матриц. 
Изучите эти реализации и выполните следующие задания:

### 1. Напишите метод `lu` для класса `Matrix`, выполняющий LU разложение.

In [4]:
def LU_func(self): # Doolittle
    l = FullMatrix.zero(self.height,self.width,0.0)
    u = FullMatrix.zero(self.height,self.width,0.0)
    for i in range(self.height):
        l[i,i] = 1
    for i in range(self.height):
        for j in range(i, self.width):
            u[i,j] = self[i,j] - np.sum(l[i,:]*u[:,j])
        for j in range(i+1, self.width):
            l[j,i] = (self[j,i] - np.sum(l[j,:]*u[:,i]))/u[i,i]
    return l, u
                
    
Matrix.lu = LU_func

m = FullMatrix.zero(5,5,0)
m[0,:]=[1,3,5,7,11]
m[1,:]=[7,3,19,7,15]
m[2,:]=[4,8,6,7,11]
m[3,:]=[5,6,3,2,1]
m[4,:]=[7,4,2,7,9]
print(m)
print('')

l, u = m.lu()
print(l)
print('')
print(u)
print('')
print(l*u)

| 1 3 5  7 11 |
| 7 3 19 7 15 |
| 4 8 6  7 11 |
| 5 6 3  2 1  |
| 7 4 2  7 9  |

| 1.0 0.0                0.0                0.0               0.0 |
| 7.0 1.0                0.0                0.0               0.0 |
| 4.0 0.2222222222222222 1.0                0.0               0.0 |
| 5.0 0.5                1.3404255319148939 1.0               0.0 |
| 7.0 0.9444444444444444 1.7127659574468088 4.850877192982451 1.0 |

| 1.0 3.0   5.0                 7.0                 11.0               |
| 0.0 -18.0 -16.0               -42.0               -62.0              |
| 0.0 0.0   -10.444444444444443 -11.666666666666668 -19.22222222222222 |
| 0.0 0.0   0.0                 3.6382978723404307  2.765957446808514  |
| 0.0 0.0   0.0                 0.0                 10.061403508771935 |

| 1.0 3.0 5.0  7.0 11.0 |
| 7.0 3.0 19.0 7.0 15.0 |
| 4.0 8.0 6.0  7.0 11.0 |
| 5.0 6.0 3.0  2.0 1.0  |
| 7.0 4.0 2.0  7.0 9.0  |


### 2. Реализуйте метод `det`, вычисляющий определитель матрицы, опираясь на LU разложение.

In [5]:
def det_func(self):
    _, u = self.lu()
    det = 1
    for i in range(self.height):
        det *= u[i,i]
    return det

Matrix.det = det_func
print(m.det())

#TODO имеет смысл сравнить со встроенным методом вычисления определителя, скорее всего это работает :) но как можно в этом убедиться, если не сравнить?
print(np.linalg.det(m.data))

6882.000000000013
6881.999999999999


Они равны! 🎉🎉

### 3. Реализация `FullMatrix` может содержать своими элементами другие матрицы, т.е. описывать блочную матрицу. Убедитесь, что ваша реализация LU разложения работает с блочными матрицами.

In [6]:
#TODO что по существу происходит в этом блоке кода? из блочной матрицы просто собирается стандартная и к ней применяется LU?

# так и есть. функция flatten делает из блочной матрицы численную матрицу,
# метод lu переопределяется и вызывает flatten (но только если матрица блочная)

@classmethod
def id_func(_cls, n, d=1):
    return FullMatrix(np.diag(n*[d]))

def flatten_func(self):
    text = [[TextBlock.from_str(f"{self[r,c]}") for c in range(self.width)] for r in range(self.height)]
    result = []
    for r in range(self.height):
        lines = TextBlock.merge(text[r][c].format() for c in range(self.width))
        for l in range(len(lines)):
            x = ''.join(f"{lines[l]}".split('|')).split()
            x = [float(y) for y in x]
            result.append(x)
    m = FullMatrix(np.array(result))
    return m

def LU_func(self): # Doolittle
    m = self.flatten() if (self.dtype == object) else self
    l = FullMatrix.zero(m.height, m.width, 0.0)
    u = FullMatrix.zero(m.height, m.width, 0.0)
    for i in range(m.height):
        l[i,i] = 1
    for i in range(m.height):
        for j in range(i, m.width):
            u[i,j] = m[i,j] - np.sum(l[i,:]*u[:,j])
        for j in range(i+1, m.width):
            l[j,i] = (m[j,i] - np.sum(l[j,:]*u[:,i]))/u[i,i]
    return l, u

FullMatrix.id = id_func
FullMatrix.flatten = flatten_func
FullMatrix.lu = LU_func

mb = FullMatrix(np.array([[m, FullMatrix.id(m.height, 2)*m],[FullMatrix.id(m.height, 3)*m, m]]))
print(mb)
print('')

l, u = mb.lu()
#print(l)
#print('')
#print(u)
#print('')
print(l*u)

| | 1 3 5  7 11 |    | 2  6  10 14 22 | |
| | 7 3 19 7 15 |    | 14 6  38 14 30 | |
| | 4 8 6  7 11 |    | 8  16 12 14 22 | |
| | 5 6 3  2 1  |    | 10 12 6  4  2  | |
| | 7 4 2  7 9  |    | 14 8  4  14 18 | |
|                                       |
| | 3  9  15 21 33 | | 1 3 5  7 11 |    |
| | 21 9  57 21 45 | | 7 3 19 7 15 |    |
| | 12 24 18 21 33 | | 4 8 6  7 11 |    |
| | 15 18 9  6  3  | | 5 6 3  2 1  |    |
| | 21 12 6  21 27 | | 7 4 2  7 9  |    |

| 1.0  3.0  5.0                7.0  11.0               2.0  6.0  10.0              14.0              22.0               |
| 7.0  3.0  19.0               7.0  15.0               14.0 6.0  38.0              14.0              30.0               |
| 4.0  8.0  6.0                7.0  11.0               8.0  16.0 12.0              14.0              22.0               |
| 5.0  6.0  3.0                2.0  1.0                10.0 12.0 6.0               4.0               2.0                |
| 7.0  4.0  2.0                7.0  9.0          

### 4. Реализуйте LUP разложение с перестановкой строк. Предъявите матрицу, на которой LUP разложение работает, а LU - нет.

In [30]:
def transpose_func(self):
    return FullMatrix(np.transpose(self.data.copy()))

def pivoting_func(self):
    p = FullMatrix(np.eye(self.height, self.width))
    a = FullMatrix(self.data.copy())
    for i in range(a.height):
        prow = i
        for j in range(i+1, a.height):
            if abs(a[j, i]) > abs(a[prow, i]):
                prow = j
        if (prow > i):
            ad = a.data
            ad[[prow,i]] = ad[[i,prow]]
            a.data = ad
            pd = p.data
            pd[[prow,i]] = pd[[i,prow]]
            p.data = pd
    return a, p.transpose()

def LUP_func(self):
    a, p = self.pivot()
    l, u = a.lu()
    return l, u, p

FullMatrix.transpose = transpose_func
FullMatrix.pivot = pivoting_func
FullMatrix.lup = LUP_func

m = FullMatrix(np.array([[1, 2, 3],
                   [0, 0, 4],
                   [5, 6, 0]]))
print(m)
print('')

l, u, p = m.lup()
print(l)
print('')
print(u)
print('')
print(p)
print('')
print(p*l*u)
print('')

l,u = m.lu() # does not work
print(l*u)

| 1 2 3 |
| 0 0 4 |
| 5 6 0 |

| 1.0 0.0 0.0 |
| 0.2 1.0 0.0 |
| 0.0 0.0 1.0 |

| 5.0 6.0                0.0 |
| 0.0 0.7999999999999998 3.0 |
| 0.0 0.0                4.0 |

| 0.0 1.0 0.0 |
| 0.0 0.0 1.0 |
| 1.0 0.0 0.0 |

| 1.0 2.0 3.0 |
| 0.0 0.0 4.0 |
| 5.0 6.0 0.0 |

| 1.0 2.0 nan |
| 0.0 0.0 nan |
| nan nan nan |


C:\Temp\ipykernel_12996\2043365585.py:27: RuntimeWarning: divide by zero encountered in scalar divide
  l[j,i] = (m[j,i] - np.sum(l[j,:]*u[:,i]))/u[i,i]
C:\Temp\ipykernel_12996\4065518629.py:97: RuntimeWarning: invalid value encountered in scalar multiply
  add = self[r,k]*other[k,c]
C:\Temp\ipykernel_12996\4065518629.py:98: RuntimeWarning: invalid value encountered in scalar add
  acc = add if acc is None else acc+add


### 5. Реализуйте метод прогонки и реализуйте метод `Matrix.solve` для решения линейных систем уравнений.

In [31]:
def check_trid_func(self): # works for square matrices
    assert self.height == self.width
    for i in range(self.height):
        for j in range(i+2, self.height):
            if ((self[i,j] != 0) | (self[j,i] != 0)):
                return False
    return True

def sweep_func(self, dd):
    assert self.check_trid()
    m = FullMatrix(self.data.copy())
    d = FullMatrix(dd.data.copy())
    for i in range(1, m.height):
        q = m[i,i-1]/m[i-1,i-1]
        m[i,i] = m[i,i]-q*m[i-1,i]
        d[i,0] = d[i,0]-q*d[i-1,0]
    x = d.empty_like()
    x[-1,0] = d[-1,0]/m[-1,-1]
    for i in reversed(range(m.height-1)):
        x[i,0] = (d[i,0]-m[i,i+1]*x[i+1,0])/m[i,i]
    return x

FullMatrix.check_trid = check_trid_func
FullMatrix.sweep = sweep_func

mtr = FullMatrix(np.array([[3,1,0,0,0],[1,3,1,0,0],[0,1,3,1,0],[0,0,1,3,1],[0,0,0,1,3]], dtype=float))
d = FullMatrix(np.array([[1],[2],[3],[4],[5]], dtype=float))

x = mtr.sweep(d)

print(d)
print('')
print(mtr)
print('')
print(x)
print('')
print(FullMatrix(np.dot(mtr.data.copy(), x.data.copy())))

| 1.0 |
| 2.0 |
| 3.0 |
| 4.0 |
| 5.0 |

| 3.0 1.0 0.0 0.0 0.0 |
| 1.0 3.0 1.0 0.0 0.0 |
| 0.0 1.0 3.0 1.0 0.0 |
| 0.0 0.0 1.0 3.0 1.0 |
| 0.0 0.0 0.0 1.0 3.0 |

| 0.20833333333333334 |
| 0.375               |
| 0.6666666666666666  |
| 0.625               |
| 1.4583333333333335  |

| 1.0 |
| 2.0 |
| 3.0 |
| 4.0 |
| 5.0 |


In [32]:
def solve_func(self, d):
    l, u = self.lu() # we will solve Ly=d and Ux=y
    
    y = d.empty_like()
    
    for i in range(y.height): # Ly=d
        y[i,0] = d[i,0] - np.dot(l[i, :i], y[:i, 0])
        
    x = d.empty_like()
    
    for i in reversed(range(x.height)): # Ux=y
        x[i,0] = (y[i,0] - np.dot(u[i, i+1:], x[i+1:,0])) / u[i, i]
    return x

Matrix.solve = solve_func

d = FullMatrix(np.array([[1],[2],[3],[4],[5]], dtype=float))
m = FullMatrix(np.array([[2, 1, 3, 4, 5],
                   [1, 4, 2, 1, 6],
                   [3, 2, 5, 3, 2],
                   [4, 1, 3, 6, 4],
                   [5, 6, 2, 4, 7]], dtype=float))

x = m.solve(d)
print(d)
print('')
print(x)
print('')
print(FullMatrix(np.dot(m.data.copy(), x.data.copy())))
print('Error: ', sum(x[0]**2 for x in (FullMatrix(np.dot(m.data.copy(), x.data.copy()))-d).data))

| 1.0 |
| 2.0 |
| 3.0 |
| 4.0 |
| 5.0 |

| -3.691056910569073   |
| 3.829268292682904    |
| -0.30081300813007966 |
| 4.097560975609727    |
| -2.186991869918685   |

| 1.0000000000000022 |
| 1.9999999999999991 |
| 3.000000000000001  |
| 3.999999999999993  |
| 5.000000000000011  |
Error:  1.705911707540438e-28


### 6. Реализуйте класс `SymmetricMatrix`, хранящий симметричные матрицы. Убедитесь, что метод `Matrix.lu` корректно работает с этим классом. Модифицируйте этот метод для класса `SymmetricMatrix` так, чтобы он использовал симметричность матрицы и работал в два раза быстрее.

In [33]:
from timeit import default_timer as timer

In [34]:
class SymmetricMatrix(FullMatrix):
    """
    Заполненная симметричная матрица
    """
    def __init__(self, data):
        """
        Создает объект, хранящий матрицу в виде np.ndarray `data`.
        """
        assert isinstance(data, np.ndarray)
        self.data = data
        assert self.height == self.width
        for i in range(self.dim):
            for j in range(i+1, self.dim):
                assert self[i,j] == self[j,i]
        
    @classmethod
    def zero(_cls, dim, default=0):
        data = np.empty((dim, dim), dtype=type(default))
        data[:] = default
        return SymmetricMatrix(data)
    
    @property
    def dim(self):
        return self.height
    
    def __setitem__(self, key, value):
        row, column = key
        self.data[row, column] = value
        self.data[column, row] = value

In [35]:
m = FullMatrix(np.array([[1, 2, 3, 4, 5],
                             [2, 6, 7, 8, 9],
                             [3, 7, 10, 11, 12],
                             [4, 8, 11, 13, 14],
                             [5, 9, 12, 14, 15]], dtype=float))
print(m)
t0 = timer()
l, u = m.lu()
t = timer()
print('')
print(l)
print('')
print(u)
print('')
print('Calculated in', t-t0, 'seconds')

| 1.0 2.0 3.0  4.0  5.0  |
| 2.0 6.0 7.0  8.0  9.0  |
| 3.0 7.0 10.0 11.0 12.0 |
| 4.0 8.0 11.0 13.0 14.0 |
| 5.0 9.0 12.0 14.0 15.0 |

| 1.0 0.0  0.0  0.0 0.0 |
| 2.0 1.0  0.0  0.0 0.0 |
| 3.0 0.5  1.0  0.0 0.0 |
| 4.0 0.0  -2.0 1.0 0.0 |
| 5.0 -0.5 -5.0 2.2 1.0 |

| 1.0 2.0 3.0 4.0  5.0                |
| 0.0 2.0 1.0 0.0  -1.0               |
| 0.0 0.0 0.5 -1.0 -2.5               |
| 0.0 0.0 0.0 -5.0 -11.0              |
| 0.0 0.0 0.0 0.0  1.2000000000000028 |

Calculated in 0.0006856999825686216 seconds


In [36]:
def ldl_func(self):
    l = FullMatrix(np.eye(self.dim))
    d = np.zeros(self.dim)
    for i in range(self.dim):
        for j in range(i):
            sum_val = sum(l[i,k] * d[k] * l[j,k] for k in range(j))
            l[i,j] = (self[i,j] - sum_val) / d[j]
        sum_val = sum(l[i,k]*l[i,k] * d[k] for k in range(i))
        d[i] = self[i,i] - sum_val
    return l, d

def sym_lu_func(self):
    l,d = self.ldl()
    dm = np.zeros((self.dim,self.dim))
    for i in range(self.dim):
        dm[i,i] = d[i]
    return l, FullMatrix(np.dot(dm, l.transpose().data))

SymmetricMatrix.ldl = ldl_func
SymmetricMatrix.lu = sym_lu_func

m = SymmetricMatrix(np.array([[1, 2, 3, 4, 5],
                             [2, 6, 7, 8, 9],
                             [3, 7, 10, 11, 12],
                             [4, 8, 11, 13, 14],
                             [5, 9, 12, 14, 15]], dtype=float))
print(m)
print('')
t0 = timer()
l, u = m.lu()
t = timer()
print(l)
print('')
print(u)
print('')
print(l*u)
print('')
print('Calculated in', t-t0, 'seconds')

| 1.0 2.0 3.0  4.0  5.0  |
| 2.0 6.0 7.0  8.0  9.0  |
| 3.0 7.0 10.0 11.0 12.0 |
| 4.0 8.0 11.0 13.0 14.0 |
| 5.0 9.0 12.0 14.0 15.0 |

| 1.0 0.0  0.0  0.0 0.0 |
| 2.0 1.0  0.0  0.0 0.0 |
| 3.0 0.5  1.0  0.0 0.0 |
| 4.0 0.0  -2.0 1.0 0.0 |
| 5.0 -0.5 -5.0 2.2 1.0 |

| 1.0 2.0 3.0 4.0  5.0                |
| 0.0 2.0 1.0 0.0  -1.0               |
| 0.0 0.0 0.5 -1.0 -2.5               |
| 0.0 0.0 0.0 -5.0 -11.0              |
| 0.0 0.0 0.0 0.0  1.2000000000000028 |

| 1.0 2.0 3.0  4.0  5.0  |
| 2.0 6.0 7.0  8.0  9.0  |
| 3.0 7.0 10.0 11.0 12.0 |
| 4.0 8.0 11.0 13.0 14.0 |
| 5.0 9.0 12.0 14.0 15.0 |

Calculated in 0.00042260007467120886 seconds


Видно, что хоть и не в 2 раза, но значительный выигрыш в скорости есть (в 1.25 раз)

P.S.: выигрыш немного варьируется, конечно, если я пересчитаю, но он всегда положителен (больше 1) и не превышает 1.5, как правило. Такое ограничение можно объяснить затратой времени на инициализацию классов, можно заметить, что numpy.ndarray инициализируется быстрее (что очевидно). Однако, безусловно, мы всем этим занимаемся, чтобы один раз инициализировать матрицу, а потом быстро и удобно все с ней считать

### 7. Как влияет симметричность матрицы на устойчивость LU разложения?

В соответствии с *Gene H. Golub, Charles F. Van Loan: "Matrix Computations" 4th edition*, разложение симметричной матрицы, происходящее через факторизацию $LDL^T$ является более устойчивым, так как сама по себе матрица $L$ накладывает доп. условия на $U=DL^T$, которые отсутствуют для обычного $LU$ разложения. Так, изначальное условие $LDL^T$ на симметричность и положительную определенность исходной матрицы результирует в повышение устойчивости разложения

### 8. Реализуйте класс `BandMatrix` для хранения ленточных матриц. Убедитесь в работоспособности методов `lu` и `solve`.

In [37]:
class BandMatrix(FullMatrix):
    """
    Ленточная матрица (квадратная)
    """
    def __init__(self, diag, upper, lower):
        assert isinstance(diag, np.ndarray)
        assert isinstance(upper, np.ndarray)
        assert isinstance(lower, np.ndarray)
        self.diag = diag
        self.upper = upper
        self.lower = lower
        self.dim = len(diag)
        self.data = np.zeros((self.dim,self.dim))
        for i in range(self.dim):
            self[i,i] = diag[i]
            for j in range(len(upper)):
                try:
                    self[i,i+j+1]=upper[j][i]
                except Exception as e:
                    ... # I will not raise AssertionError if provided subdiagonal arrays are too big
                    # redundant element will be merely dropped
                    # Here I also do not check if upper is an array of arrays 
            for j in range(len(lower)):
                try:
                    self[i+j+1,i]=lower[j][i]
                except Exception as e:
                    ... # I will not raise AssertionError if provided subdiagonal arrays are too big
                    # redundant element will be merely dropped
    
    @property
    def dim(self):
        return self._dim

    @dim.setter
    def dim(self, value):
        self._dim = value
    
    @property
    def diag(self):
        return self._diag

    @diag.setter
    def diag(self, value):
        self._diag = value
    
    @property
    def upper(self):
        return self._upper

    @upper.setter
    def upper(self, value):
        self._upper = value
    
    @property
    def lower(self):
        return self._lower

    @lower.setter
    def lower(self, value):
        self._lower = value

In [48]:
bm = BandMatrix(np.array([11,22,33,44,55]),np.array([[2,3,4,5],[6,7,8]],dtype=object),np.array([[5,4,0,3]],dtype=object))
print(bm,'\n')

l, u = bm.lu()
print(l*u,'\n')
d = FullMatrix(np.array([[1],[2],[3],[4],[5]], dtype=float))
x = bm.solve(d)
print(x,'\n')
print(FullMatrix(np.dot(bm.data.copy(), x.data.copy())),'\n')
print('Error: ', sum(x[0]**2 for x in (FullMatrix(np.dot(bm.data.copy(), x.data.copy()))-d).data))

| 11.0 2.0  6.0  0.0  0.0  |
| 5.0  22.0 3.0  7.0  0.0  |
| 0.0  4.0  33.0 4.0  8.0  |
| 0.0  0.0  0.0  44.0 5.0  |
| 0.0  0.0  0.0  3.0  55.0 | 

| 11.0 2.0  6.0  0.0  0.0  |
| 5.0  22.0 3.0  7.0  0.0  |
| 0.0  4.0  33.0 4.0  8.0  |
| 0.0  0.0  0.0  44.0 5.0  |
| 0.0  0.0  0.0  3.0  55.0 | 

| 0.052836352836352844 |
| 0.04565955280240994  |
| 0.05458016886588315  |
| 0.08108108108108109  |
| 0.08648648648648649  | 

| 1.0 |
| 2.0 |
| 3.0 |
| 4.0 |
| 5.0 | 

Error:  0.0


### 9. Воспользуйтесь реализованными классами для решения уравнения Пуассона $\Delta f=g$

В одномерном случае задача Пуассона:


$$
\displaystyle{ \Delta _{ 1 } f ^{ 1 } = g ^{ 1 } } \quad \Leftrightarrow \quad
\displaystyle{ \left[ \begin{array}{cccccc} - 2 & 1 & 0 & \cdots & 0 & 1 \\ 1 & - 2 & 1 & 0 & \cdots & 0 \\ 0 & 1 & - 2 & 1 & \ddots & \vdots \\ \vdots & \ddots & \ddots & \ddots & \ddots & 0 \\ 0 & \cdots & 0 & 1 & - 2 & 1 \\ 1 & 0 & \cdots & 0 & 1 & - 2 \end{array} \right] \left[ \begin{array}{c} f _{ 1 } \\ \vdots \\ f _{ n } \end{array} \right] = \left[ \begin{array}{c} g _{ 1 } \\ \vdots \\ g _{ n } \end{array} \right] }
$$

In [59]:
n = 20
pp1d = BandMatrix(np.array([-2]*n), np.array([[1]*(n-1)]), np.array([[1]*(n-1)]))
pp1d[-1,0] = 1
pp1d[0,-1] = 1
print(pp1d,'\n')

g = FullMatrix(np.array([[np.cos(np.pi*x)] for x in np.linspace(0,1,n)])) # periodic boundary condition

f = pp1d.solve(g)
print(f)

| -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  |
| 1.0  -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  |
| 0.0  1.0  -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  |
| 0.0  0.0  1.0  -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  |
| 0.0  0.0  0.0  1.0  -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  |
| 0.0  0.0  0.0  0.0  1.0  -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  |
| 0.0  0.0  0.0  0.0  0.0  1.0  -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  |
| 0.0  0.0  0.0  0.0  0.0  0.0  1.0  -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  |
| 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  -2.0 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  |
| 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  -2.0 1.0  0.0  0.